In [131]:
import sys
sys.path.append("./../..")

In [132]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [133]:
import torch
import torch.nn as nn
import numpy as np

#
from effcn.models_mnist import CapsNet
from misc.utils import count_parameters

### receptive firld

In [165]:
def rec_fld(L):
    L = np.array(L)
    r = 1
    for i, la in enumerate(L):
        r += (la[0] - 1)*np.prod(L[0:i+1,1])
    return r

In [197]:

l1 = [9,2]
l2 = [9,2]


L = [l1,l2]

L = rec_fld(L)

L

49

In [262]:
class BackboneHinton(nn.Module):
    """
        Backbone model from Efficient-CapsNet for MNIST
    """

    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
                        nn.Conv2d(1, 256, kernel_size=9, stride=1),
                        #nn.Conv2d(256, 32*8, kernel_size=9, stride=2, groups=256),
        )

    def forward(self, x):
        """
            IN:
                x (b, 1, 28, 28)
            OUT:
                x (b, 128, 9, 9)
        """
        return self.layers(x)

In [263]:
T = BackboneHinton()
print(count_parameters(T))
x = torch.rand(1,1,28,28)

T(x).shape

#5329664

20992


torch.Size([1, 256, 20, 20])

In [208]:
CN = CapsNet()

In [209]:
a = CN.backbone.layers[0]
b = CN.primcaps.conv

a,b

a.kernel_size, a.stride
b.kernel_size, b.stride



((9, 9), (2, 2))

In [210]:
A = nn.Conv2d(in_channels=2, out_channels=3,kernel_size=5,stride=2,padding=0, bias=True)

count_parameters(A), 3*2*(5**2)+3, calc_params(ch_in=2, ch_out=3, kernel=5, bias=True)



(153, 153, 153)

In [211]:
def calc_params(ch_in, ch_out, kernel, bias=True):
    out = ch_in*ch_out*kernel**2
    if bias == True:
       out = out + ch_out
    return out

##### EFFCN

In [252]:
bb_effcn = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=(5, 5), padding="valid"),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, kernel_size=(5, 5), padding="valid"),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=(5, 5), padding="valid"),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=1, padding="valid"),
            nn.Conv2d(128, 128, kernel_size=9, stride=1, groups=1, padding="valid")
        )

In [253]:
print(count_parameters(bb_effcn))

x = torch.rand(1,1,28,28)

bb_effcn(x).shape

1555968


torch.Size([1, 128, 6, 6])

In [214]:
130432-130112, 130432-130304


228736
1555968

(320, 128)

In [226]:
L_ef= [[5,1],[3,1],[3,1],[3,2],[9,1]]

rec_fld(L_ef)

29

##### EM Caps

In [216]:
bb_em = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(5, 5), stride=2, padding=2),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(1, 1), stride=1, padding="valid"),
        )



In [232]:
count_parameters(bb_em)
x = torch.rand(1,1,28,28)

bb_em(x).shape

torch.Size([1, 32, 14, 14])

In [218]:
L_em= [[5,2],[1,1]]

rec_fld(L_em)

9

In [230]:
CNnew = CapsNet()

CNnew

CapsNet(
  (backbone): BackboneHinton(
    (layers): Sequential(
      (0): Conv2d(1, 256, kernel_size=(9, 9), stride=(1, 1))
      (1): ReLU(inplace=True)
      (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (primcaps): PrimaryCapsLayer(
    (conv): Conv2d(256, 256, kernel_size=(9, 9), stride=(2, 2))
  )
  (digitcaps): CapsLayer(
    (routing_module): AgreementRouting()
  )
  (decoder): Decoder(
    (layers): Sequential(
      (0): Linear(in_features=160, out_features=512, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=512, out_features=1024, bias=True)
      (3): ReLU(inplace=True)
      (4): Linear(in_features=1024, out_features=784, bias=True)
      (5): Sigmoid()
    )
  )
)

In [299]:
bb_stuff = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=(5, 5), padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, kernel_size=(5, 5), padding=0),
            #nn.ReLU(inplace=True),
            #nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=(3, 3), padding=1),
            #nn.ReLU(inplace=True),
            #nn.BatchNorm2d(64),
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=1, padding="valid"),
            nn.Conv2d(128, 128, kernel_size=9, stride=2, groups=1, padding="valid")
        )

In [300]:
print(count_parameters(bb_stuff))
x = torch.rand(1,1,28,28)

bb_stuff(x).shape

1490176


torch.Size([1, 128, 6, 6])

In [301]:
L_em= [[5,1],[5,1],[3,1],[3,1],[9,2]]

rec_fld(L_em)

29